In [17]:
%matplotlib qt

import matplotlib.pyplot as plt
import scipy.optimize as opt
import pandas as pd 
import numpy as np 
import cv2

In [18]:
# paths
df_false_positives = pd.read_csv("/media/ivan/Ivan/data_20_2_24/data_false_positives_production_day_first_condition.csv")
df_small_red = pd.read_csv("/media/ivan/Ivan/data_20_2_24/data_small_red.csv")
df_big_red_meat = pd.read_csv("/media/ivan/Ivan/data_20_2_24/data_big_red_on_meat.csv")
df_big_red_fat = pd.read_csv("/media/ivan/Ivan/data_20_2_24/data_big_red_on_fat.csv")

# take out first column (positions)
df_false_positives = df_false_positives.drop(df_false_positives.columns[0], axis='columns')  
df_small_red = df_small_red.drop(df_small_red.columns[0], axis='columns')
df_big_red_meat = df_big_red_meat.drop(df_big_red_meat.columns[0], axis='columns')
df_big_red_fat = df_big_red_fat.drop(df_big_red_fat.columns[0], axis='columns') 

In [19]:
# shapes/sizes of data
print("============ Data ============")
print(f'False positives: {df_false_positives.shape}')
print(f'Small red: {df_small_red.shape}')
print(f'Big red on meat: {df_big_red_meat.shape}')
print(f'Big red on fat: {df_big_red_fat.shape}')

============ Data ============
False positives: (9622, 184)
Small red: (7820, 184)
Big red on meat: (66798, 184)
Big red on fat: (59152, 184)


In [20]:
# We take points/spectral bands of our interest
df_false_positives_m = df_false_positives.iloc[:, [24,79,153]]
df_small_red_m = df_small_red.iloc[:, [24,79,153]]
df_big_red_meat_m = df_big_red_meat.iloc[:, [24,79,153]]
df_big_red_fat_m = df_big_red_fat.iloc[:, [24,79,153]]

# df_false_positives_m = df_false_positives.iloc[:, 0:25].join(df_false_positives.iloc[:, [79,153]])
# df_small_red_m = df_small_red.iloc[:, 0:25].join(df_small_red.iloc[:, [79,153]])

# into numpy arrays (only points of interest)
false_positives = np.array(df_false_positives_m)
small_red = np.array(df_small_red_m)
big_red_meat = np.array(df_big_red_meat_m)
big_red_fat = np.array(df_big_red_fat_m)

In [21]:
def f(x, data_array):
    
    # Number of samples (data)
    m = len(data_array)
    
    for i in range(len(data_array)):
        
        # Values of each point at each iteration
        value25, value80, value154 = data_array[i]
    
        # Operation 
        calc = (1/2*m)*(pow((value154*x[0]-value25),2)+pow((value154*x[1]-value80),2))

        return calc

In [22]:
start = np.array([0, 0])

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_cg.html
result_g = opt.fmin_cg(f, start, args=(small_red,))
print(result_g)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 15
         Gradient evaluations: 5
[4.15137437 3.10282845]


In [23]:
# # Entire spectrum into numpy arrays
# false_positives_e = np.array(df_false_positives)
# small_red_e = np.array(df_small_red)
# big_red_meat_e = np.array(df_big_red_meat)
# big_red_fat_e = np.array(df_big_red_fat)

# # for larger datasets --> taking 10k random samples
# random_big_red_meat = np.array(df_big_red_meat.sample(n=10000))
# random_big_red_fat = np.array(df_big_red_fat.sample(n=10000))

# for _, sample_signal in enumerate(random_big_red_fat):
#     plt.plot(sample_signal)

# plt.title('Big red on fat (10k random samples)')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.legend()
# plt.show()

In [24]:
# Check if it works
y1, y2 = result_g
samples = false_positives.shape[0]
count = 0

for i in range(samples):
    value25, value80, value154 = false_positives[i]
    test = (1/4)*(pow((value154*y1-value25), 2) + pow(value154*y2-value80, 2))
    # print(test)
    if test < 0.001:
        count += 1

print(count)

1886
